## Install JDK
## Install Spark
## Set Environment variables
## Create a Spark Session

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark-2.4.3-bin-hadoop2.6/
spark-2.4.3-bin-hadoop2.6/python/
spark-2.4.3-bin-hadoop2.6/python/setup.cfg
spark-2.4.3-bin-hadoop2.6/python/pyspark/
spark-2.4.3-bin-hadoop2.6/python/pyspark/resultiterable.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/heapq3.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/join.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/version.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/rdd.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/java_gateway.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/find_spark_home.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/_globals.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/worker.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/accumulators.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/mllib/
spark-2.4.3-bin-hadoop2.6/python/pyspark/mllib/feature.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/mllib/random.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/mllib/recommendation.py
spark-2.4.3-bin-hadoop2.6/python/pyspark/mllib/fpm.py
spark-2.4.3-bin-hadoop2.6/python/py

## Copy a data file to your local Colab environment

In [2]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv

--2020-04-27 08:42:09--  https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2020-04-27 08:42:09 (64.9 MB/s) - ‘bank_prospects.csv’ saved [306/306]



## Check if the file is copied

In [3]:
!ls


bank_prospects.csv  spark-2.4.3-bin-hadoop2.6
sample_data	    spark-2.4.3-bin-hadoop2.6.tgz


# DataFrame

## Read the CSV file into a DataFrame

In [0]:
bankProspectsDF = spark.read.csv("bank_prospects.csv",header=True)

In [5]:
bankProspectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



## Remove the record with unknow value in country column

In [0]:
bankProspectsDF1 = bankProspectsDF.filter(bankProspectsDF['country'] != "unknown")

In [7]:
bankProspectsDF1.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



##  Cast the String datatype to Integer/Float

In [8]:
bankProspectsDF1.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType,FloatType

In [0]:
bankProspectsDF2 = bankProspectsDF1.withColumn("age", bankProspectsDF1["age"].cast(IntegerType())).withColumn("salary", bankProspectsDF1["salary"].cast(FloatType()))


In [11]:
bankProspectsDF2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Replace Age and Salary with average values of their respective column

import mean from sql.fuctions

In [0]:
from pyspark.sql.functions import mean

### Calculate "mean" value of the age

In [0]:
mean_age_val = bankProspectsDF2.select(mean(bankProspectsDF2['age'])).collect()

In [14]:
type(mean_age_val)

list

In [15]:
mean_age_val

[Row(avg(age)=22.11111111111111)]

In [0]:
mean_age = mean_age_val[0][0]

In [17]:
mean_age

22.11111111111111

### Calculate mean salary value

In [0]:
mean_salary_val = bankProspectsDF2.select(mean(bankProspectsDF2['salary'])).collect()

In [0]:
mean_salary = mean_salary_val[0][0]

In [20]:
mean_salary

31875.0

### Replace missing age with average value

In [21]:
bankProspectsDF2.show()

+----+-------+------+-------+---------+
| age| salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   null|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   null|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|null|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+



In [0]:
bankbankProspectsDF3 = bankProspectsDF2.na.fill(mean_age,["age"])

In [23]:
bankbankProspectsDF3.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   null|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   null|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



### Replace missing age with salary value

In [0]:
bankbankProspectsDF4 = bankbankProspectsDF3.na.fill(mean_salary,["salary"])

In [25]:
bankbankProspectsDF4.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [26]:
bankbankProspectsDF4.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Write the transformed file to a new csv file 

In [0]:
bankbankProspectsDF4.write.format("csv").save("bank_prospects_transformed")

In [28]:
!ls

bank_prospects.csv	    spark-2.4.3-bin-hadoop2.6
bank_prospects_transformed  spark-2.4.3-bin-hadoop2.6.tgz
sample_data


In [29]:
!ls bank_prospects_transformed/

part-00000-457e6249-a16a-468f-988a-149fddea139f-c000.csv  _SUCCESS


In [31]:
!cat bank_prospects_transformed/part-00000-457e6249-a16a-468f-988a-149fddea139f-c000.csv

18,20000.0,Male,Germany,N
19,22000.0,Female,France,N
20,24000.0,Female,England,N
21,31875.0,Male,England,N
22,50000.0,Male,France,Y
23,35000.0,Female,England,N
24,31875.0,Male,Germany,N
25,32000.0,Female,France,Y
22,35000.0,Male,Germany,N
27,37000.0,Female,France,N
